<a href="https://colab.research.google.com/github/tomvangasse/game-store-manager/blob/main/tomsMenuProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
Upload files here
Click run and then click choose files:

    Video_Game_Info.txt
    Board_Game_Info.txt
    Subscription_Info.txt
    Game_feedback.txt
    Booking.txt
    Rental.txt
    feedbackManager.pyc
    subscriptionManager.pyc
'''


from google.colab import files
uploaded = files.upload()

Saving Board_Game_Info.txt to Board_Game_Info.txt
Saving Booking.txt to Booking.txt
Saving Game_Feedback.txt to Game_Feedback.txt
Saving Rental.txt to Rental.txt
Saving Subscription_Info.txt to Subscription_Info.txt
Saving Video_Game_Info.txt to Video_Game_Info.txt
Saving feedbackManager.pyc to feedbackManager.pyc
Saving subscriptionManager.pyc to subscriptionManager.pyc


In [4]:
import subscriptionManager as smTV
import feedbackManager as fbTV

sub = smTV.load_subscriptions()

print(sub)


{'abcd': {'SubscriptionType': 'basic', 'StartDate': datetime.datetime(2025, 10, 10, 0, 0), 'EndDate': datetime.datetime(2030, 10, 10, 0, 0)}}


Register / Sign in

In [5]:
import subscriptionManager as smTV
import datetime
from dateutil.relativedelta import relativedelta as rd

subscriptions = smTV.load_subscriptions()


while True:
    userID = input("Enter user ID to sign up / register: ")

    if len(userID) != 4:
        print("Invalid user ID: must be 4 characters long")

    else:
        break

if smTV.check_subscription(userID, subscriptions):
    print(f"Welcome, {userID}")

else:
    print("Unregistered account, please register now:")
    while True:
        subscriptionType = input("Enter subscription type (Basic or premium): ").lower()
        if subscriptionType != "basic" and subscriptionType != "premium":
            print("Invalid subscription type")
        else:
            break
    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    endTime = (date + rd(years=3)).strftime("%Y-%m-%d")

    with open("Subscription_Info.txt", mode = "a") as file:
        file.write(f"\n{userID},{subscriptionType},{startTime},{endTime}")

    print(f"Welcome, {userID}")


Enter user ID to sign up / register: tomvangasse
Invalid user ID: must be 4 characters long
Enter user ID to sign up / register: stve
Welcome, stve


Game search

In [12]:
import csv
def getVideoDict():
    with open("Video_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def getBoardDict():
    with open("Board_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def printGames(gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        for i in boardDict:
            currentGame = boardDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    elif gameType == "v":
        for i in videoDict:
            currentGame = videoDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    else:
        return

def printGameInfo(GameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        gameInfo = boardDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    elif gameType == "v":
        gameInfo = videoDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    else:
        return


def gameSearch():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    gameType = input('''Board games (b)
Video games (v)
Quit (q)
''')

    if gameType == "q":
        return False

    gameSelection = input('''Print games (press enter)
Select a specific game (enter GameID)
''')
    gameSelection
    if gameSelection == "":
        printGames(gameType)
    elif gameSelection == "q":
        return False
    else:
        printGameInfo(gameSelection, gameType)

while True:
    if gameSearch() == False:
        break


Board games (b)
Video games (v)
Quit (q)
v
Print games (press enter)
Select a specific game (enter GameID)

tloz01: The Legend of Zelda       Available / Unavailable: 5 / 0
elrn01: Elden Ring                Available / Unavailable: 5 / 0
hali01: Halo Infinite             Available / Unavailable: 5 / 0
stva01: Stardew Valley            Available / Unavailable: 5 / 0
mine01: Minecraft                 Available / Unavailable: 5 / 0
gdow01: God of War                Available / Unavailable: 5 / 0
spmr01: Super Mario               Available / Unavailable: 5 / 0
cbpk01: Cyberpunk 2077            Available / Unavailable: 5 / 0
fzhz01: Forza Horizon 5           Available / Unavailable: 5 / 0
amcs01: Animal Crossing           Available / Unavailable: 5 / 0


Board games (b)
Video games (v)
Quit (q)
q


Game rent

In [ ]:
def rentGame():
    selectedGame = input('''Enter the GameID for the desired game:
''')

#with open("Rental.txt", mode="a") as file:
    #file.write(f"\n{gameID},{Rental date}

rentGame()